<h1>Exploiting Genetic Algorithms for Optimisation </h1>
<br>

<h2> The problem</h2>
We need to make cylindrical containers with fixed volume 30 cubic cm so that, given the diameter you can figure out its height, minimizing the building material. 
This mathematically simple minimization problem could be solved with a genetic algorithm driven by the diameter (or radius) that must always be positive. The rest of the problem definition is up to you. 

In [242]:
#!pip install -r requirements.txt

In [243]:
# Install dependencies
import math
import numpy as np
import time
from tqdm.notebook import tqdm
import pickle
import random

In [244]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

## Defining the Problem

We need to optimise the building material of Cilinders of Volume = 30 cm<sup>3</sup>

$$
\begin{array}{rcl}
    Volume & = & \pi r^2 h\\
    h & = & \frac{30}{\pi r^2}\\
    r & = & x
\end{array}
\newline
$$

In order to minimise the used material, we need to minimise the surface area. 
Thus we get the following equation, for $r > 0$:
$$
\newline
\begin{array}{rcl}
    Area & = & 2\pi r^2 + 2\pi h\\
    & = & 2 \pi r^2 + \frac{60}{r}\\
\end{array}
$$

Then, our fitness equation is:
$$
\begin{array}{rcl}
fitness & = & \dfrac{1}{Area}\\
\\
fitness & = & \dfrac{1}{2 \pi r^2 + \frac{60}{r}}

\end{array}
$$



In [245]:

#1. Initial Hyperparameters
np.random.seed(2023)

#2. Define height given diameter
def height (d):
    ''' Height Function - calculates height given a radius
    Inputs:
        d   :   Diameter -> takes a number (int, float) 

    Outputs: 
        h   :   Height -> outputs the height (float) 
    '''
    r = d/2
    pi = math.pi
    #volume = pi * (r**2) * h
    h = 30/(pi*(r**2))
    return h

#3. Define Fitness
def fitness (r):
    #r=d/2
    area = (2 * (math.pi) * (r**2) + (60/r))
    fitness = 1/area
    return fitness
    
#3. Define selection methods
def ranked_fitness (current_generation, max_radius, n):
    ranked_solutions = []
    for el in current_generation:
        #print('element of ranked fitness', el)
        ranked_solutions.append([fitness(decode_r(el, max_radius, n)),decode_r(el, max_radius, n), el])
        #added a second element to keep track of solutions and not only fitness
    ranked_solutions.sort(reverse = True)
    return ranked_solutions

#4. Define selection methods
def random_choice (generation, max_radius, n, probabilities, gen_fitness, decoded_gen):
    choice = generation[np.random.randint(0, len(generation)-1)]
    #print('randomly chose: ', choice)
    return choice

def ranked_probabilities(pop_size):
    #based on https://stackoverflow.com/questions/20290831/how-to-perform-rank-based-selection-in-a-genetic-algorithm
    probabilities = []
    amax = 1.2
    amin = 2 - amax
    m = pop_size
    for i in range(pop_size):
        p = (amax - (amax-amin)*(i/(m-1)))*(1/m)
        probabilities.append(p)
    sum_probs = sum(probabilities)
    probabilities[0] = probabilities[0] + (1 - sum_probs)
    #print('Prob matrix len: ', len(probabilities))
    return probabilities

def ranked_choice (generation, max_radius, n, probabilities, gen_fitness, decoded_gen):
    gen_fit = generation
    #sorting
    choice= gen_fit[np.random.choice(np.arange(len(gen_fit)), p=probabilities)]
    return choice

def wheel_selection (generation, max_radius, n, probabilities, gen_fitness, decoded_gen):
    #create list with radius and its fitness        
    # computes the total the generation fitness
    generation_fitness = sum([el for el in gen_fitness])
    # computes the probability for each radius
    probabilities = [el/generation_fitness for el in gen_fitness]
    sum_probs = sum(probabilities)
    #if sum_probs < 1:
    #    probabilities[0] = probabilities[0] + (1 - sum_probs)
    # selects one chromosome based on the computed probabilities
    choice= generation[np.random.choice(np.arange(len(generation)), p=probabilities)]
    return choice


#5. Define Reproduction methods    
def crossover(parentA, parentB):
    #print(parentA)
    #print(parentB)
    cross_point = np.random.randint(len(parentA))
    child = parentA[:cross_point]+parentB[cross_point:]
    return child

#Mutations
def mutation(parent, probability):
    flipped = lambda x: '1' if x == '0' else '0'
    chars = (flipped(sym) if random.random() < probability else sym for sym in parent)
    return ''.join(chars)
    

#6. Reproduction Behaviour
def reproduction (current_gen, max_radius, n, sel_function, ranked_probabilities, gen_fitness, decoded_gen, mutation_probability):
    #current_gen = (np.array(current_gen).transpose())[1]
    new_generation = []
    for i in range(len(current_gen)):
        parentA = sel_function(current_gen, max_radius, n, ranked_probabilities, gen_fitness, decoded_gen)
        parentB = sel_function(current_gen, max_radius, n, ranked_probabilities, gen_fitness, decoded_gen)
        child = crossover(parentA, parentB)
        child=mutation(child, mutation_probability)# 1 mutation in 1000
        if '1' in child:
            new_generation.append(child)
        else:
            new_generation.append(parentA)
    return new_generation

        
#Function to decode binary radius
def decode_r(r, max_radius, n):
    rad=list(map(int, str(r)))
    dec_r=(max_radius/(2**n-1))*sum(rad[i]*2**i for i in range(len(rad)))
    return dec_r

#Generation zero binary
def first_gen(pop_size,n):
    gen_zero=[]
    for i in range(pop_size):
        x = []
        while 1 not in x:
            x = [np.random.choice([0, 1]) for _ in range(n)]   
        el = ''.join(map(str, x))
        gen_zero.append(el)

    return gen_zero

In [246]:
#5. New generation iterations
def digi_evolve(pop_size, max_generations, max_radius, n, sel_function_tuple, ranked_probabilities, all_gens, mutation_probability):
    #create temporary vars
    sel_function = sel_function_tuple[0]
    sel_function_name = sel_function_tuple[1]
    fittest = []
    curr_generation = first_gen(pop_size, n) #initialising first gen

    #GA begins
    #print('Evolutionary period begins')
    for i in range(max_generations):
        ranked_fit = ranked_fitness(curr_generation, max_radius, n)
        fittest.append([ranked_fit[0], i])
        for element in ranked_fit:
            #print('###################### new var', element)
            all_gens['radius'].append(element[1])
            all_gens['fitness'].append(element[0])
            all_gens['generation'].append(i)
            all_gens['selection_type'].append(sel_function_name)
            all_gens['pop_size'].append(pop_size)
            all_gens['mutation'].append(mutation_probability)
        #reproduction
        curr_generation = (np.array(ranked_fit).transpose())
        gen_fitness = np.array(curr_generation[0], dtype= float)
        gen_decoded = np.array(curr_generation[1], dtype= float)
        curr_generation = reproduction(curr_generation[2], max_radius, n, sel_function, ranked_probabilities,gen_fitness, gen_decoded, mutation_probability)
        
    #print('Evolutionary period ended')
    return fittest, all_gens


## Solution

Solving by hand, we arrive at the following results:

$$
\begin{array}{rcl}
fitness & = & \dfrac{1}{2 \pi r^2 + \frac{60}{r}}\\
\dfrac{d(fitness)}{dr} & = & 4 \pi r - \frac{60}{r^2} > 0\\
4 \pi r & > & \dfrac{60}{r^2}\\\\
4 \pi & > & \dfrac{60}{r^3}\\\\
\dfrac{4 \pi}{60} & > & \dfrac{1}{r^3}\\
\end{array}
$$

where we get:

$$
\begin{array}{rcl}
 r^3 & > & \dfrac{60}{4 \pi} = \dfrac{15}{\pi}
\end{array}
$$

finally:

$$
\begin{array}{rcl}
r_{att} & = & \sqrt[3]{\dfrac{15}{\pi}} = 1.6841
\end{array}
$$

In [247]:
#main
pop_size = 100
pop_range_steps = 10
max_generations = 500
#gen_range_steps = 50

#mutation
mutation_list = [0,1, 10]
mutation_rate = 0.001


n=8 #step:0.1176 if we work in range [0,30]
max_radius=30
selection_methods = [[random_choice, 'Random Selection'], [ranked_choice, 'Ranked Selection'],[wheel_selection, 'Wheel Selection']]

best_overall = []
all_generations = []

#progress bar setup
# Custom format with ANSI escape codes for dark green color
#dark_green = "\033[1;32;40m"
#bar_format = f"{dark_green}{{l_bar}}{{bar:50}} [{{elapsed}}]{{r_bar}}"

#initialising dict for all vals
all_gens_dict = {
        'radius'        :   [],
        'generation'    :   [],
        'fitness'       :   [], 
        'selection_type':   [], 
        'pop_size'      :   [],
        'mutation'      :   []    
    }

start_total = time.time()
start_local = time.time()

for mutation_el in mutation_list:
    mutation_probability = mutation_el * mutation_rate
    for selectionMethod in tqdm(selection_methods, desc='Selection Method'):
        print(f'Selection method: {selectionMethod[1]}')
        for pop_length in tqdm(range(pop_range_steps,pop_size+1, 10), desc='Population iterations'):
            ranked_probability = ranked_probabilities(pop_length)
            #for no_gens in tqdm(range (gen_range_steps, max_generations, 50),desc='Max Generations iterations'):
            best_of_each_gen, all_gens = digi_evolve(pop_length, max_generations, max_radius, n, selectionMethod, ranked_probability, all_gens_dict, mutation_probability)

        print('The best radius is:', max(best_of_each_gen)[0][1], 'in generation: ',max(best_of_each_gen)[1])

        best_overall.append([max(best_of_each_gen), selectionMethod[1]])
    

        end_local = time.time()
        print('Time in seconds to complete iteration of', selectionMethod[1] ,end_local - start_local, '\n')
        start_local = time.time()


end_total = time.time()
print('Time in seconds to complete entire cycle:',end_total - start_total, '\n')

print('The best radius is:', max(best_overall))




Selection Method:   0%|          | 0/3 [00:00<?, ?it/s]

Selection method: Random Selection


Population iterations:   0%|          | 0/10 [00:00<?, ?it/s]

The best radius is: 1.6470588235294117 in generation:  28
Time in seconds to complete iteration of Random Selection 6.3710548877716064 

Selection method: Ranked Selection


Population iterations:   0%|          | 0/10 [00:00<?, ?it/s]

...

The best radius is: 1.6470588235294117 in generation:  489
Time in seconds to complete iteration of Random Selection 5.989897012710571 

Selection method: Ranked Selection


Population iterations:   0%|          | 0/10 [00:00<?, ?it/s]

The best radius is: 1.6470588235294117 in generation:  499
Time in seconds to complete iteration of Ranked Selection 14.955894947052002 

Selection method: Wheel Selection


Population iterations:   0%|          | 0/10 [00:00<?, ?it/s]

The best radius is: 1.6470588235294117 in generation:  493
Time in seconds to complete iteration of Wheel Selection 23.02313804626465 

Time in seconds to complete entire cycle: 130.61163711547852 

The best radius is: [[[0.01870081198259601, 1.6470588235294117, '01110000'], 499], 'Ranked Selection']


In [248]:
#Save object for Data Analysis
fileObj = open('data_algos.pkl', 'wb')
pickle.dump(all_gens,fileObj)
fileObj.close()
print('Data saved successfully')

Data saved successfully


In [249]:
#Testing with different n
bits=range(7,15,1)
pop_size=10
max_generation=10
max_radius=30
results=[]
for k in bits:
    best_of_each_gen = digi_evolve(pop_size, max_generations, max_radius, k, [wheel_selection, 'Wheel Selection'])
    results.append(max(best_of_each_gen[0]))
print(results)

TypeError: digi_evolve() missing 3 required positional arguments: 'ranked_probabilities', 'all_gens', and 'mutation_probability'